In [103]:
import pandas as pd
import numpy as np


In [52]:
data = pd.read_csv('/home/guilherme/Documentos/repos/ds_ao_dev/projeto_webscraping/df_raw.csv')

In [53]:
data.head()

,Unnamed: 0,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
0,0,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Shell: Cotton 99%, Spandex 1%"
1,1,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Pocket lining: Polyester 65%, Cotton 35%"
2,2,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Shell: Cotton 99%, Spandex 1%"
3,3,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Pocket lining: Polyester 65%, Cotton 35%"
4,4,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Denim blue,"Shell: Cotton 99%, Spandex 1%"


In [27]:
#data['Composition']= data['Composition'].where(~data['Composition'].str.contains('Pocket lining:',na=False))

In [54]:
data = data.drop(['Unnamed: 0' ],axis=1)
data.sample()

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
85,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Black,"Pocket lining: Polyester 65%, Cotton 35%"


In [77]:
#product id

data = data.dropna(subset=['product_id'])

#product category

#product name
data['products_name']= data['products_name'].apply(lambda x: x.replace(' ','_').lower())
#products_price

#já fiz o tratamento anteriomente

#product datetime

data['scrapy_datetime']= pd.to_datetime(data['scrapy_datetime'])
#style id

#color id

#color name
data['color_name']= data['color_name'].apply(lambda x: x.replace(' ','_').replace('/','_').lower() if pd.notnull(x) else x)

#Composition

data['Composition']= data['Composition'].where(~data['Composition'].str.contains('Pocket lining:',na=False) ) 

data['Composition']= data['Composition'].where(lambda x: ~x.str.contains('Shell:',na=False) )

data['Composition'] = data['Composition'].where(lambda x: ~x.str.contains('Lining:',na=False) )


#break composition by comma (separando as composições dos materiais por coluna)
df1 = data['Composition'].str.split(',',expand=True)


#Cotton| Spandex | Polyester |  Elastomultiester

data.head()

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
0,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,light_denim_blue,NaN
1,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,light_denim_blue,NaN
2,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,light_denim_blue,NaN
3,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,light_denim_blue,NaN
4,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,denim_blue,NaN


In [221]:
#Cotton| Spandex | Polyester |  Elastomultiester
#2 etapa
#definir as colunas (organizando as colunas por máteria-prima)

#criando Dataframe vazio com o mesmo tamanho do dataframe original para prencher com as respectivas composições
df_ref = pd.DataFrame(index=np.arange(len(data) ),columns=['cotton','spandex','polyester','elastomultiester'])

#cotton
#pegando a coluna da composição cotton
df_cotton = df1[0]

#colocando o nome na coluna 
df_cotton.name = 'cotton'
#mesclando com a tabela vazia 
df_ref = pd.concat([df_ref,df_cotton],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#polyester 

#adicionando as coluna da composição Polyester
df_polyester =df1[1].where(lambda x: x.str.contains('Polyester',na=True))

#renomeando a coluna
df_polyester.name= 'polyester'

df_ref= pd.concat([df_ref,df_polyester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#Elastomultiester

#coletando apenas a composição elastano
df_elastomultiester =df1[1].where(lambda x: x.str.contains('Elastomultiester',na=True))

#renomenado a coluna eslastano
df_elastomultiester.name= 'elastomultiester'

#mesclando com df_ref vazio e com a mesma quantidade de linhas que há no index
df_ref= pd.concat([df_ref,df_elastomultiester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#Spandex

#coletando composição spandex da segunda coluna df1

df_spandex_1= df1[1].where(lambda x: x.str.contains('Spandex',na=True))

df_spandex_1.name = 'spandex'

df_ref= pd.concat([df_ref,df_spandex_1],axis=1)

df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#coletando composição spandex da terceira coluna do df1

df_spandex_2= df1[2].where(lambda x: x.str.contains('Spandex',na=True))
             
df_spandex_2.name= 'spandex'

df_ref= pd.concat([df_ref,df_spandex_2],axis=1)

df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]



#final join

data= pd.concat([data,df_ref],axis=1)


In [244]:
data.sample(35)

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition,cotton,polyester,elastomultiester,spandex
1369,985159004,men_jeans_skinny,skinny_jeans,19.99,2022-04-24 11:00:00-21:30,985159,4,light_denim_blue,NaN,NaN,None,None,None
3386,985159008,men_jeans_skinny,skinny_jeans,19.99,2022-04-24 11:00:00-21:30,985159,8,dark_gray,NaN,NaN,None,None,None
6901,938875008,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-24 11:00:00-21:30,938875,8,light_denim_blue,"Cotton 99%, Spandex 1%",Cotton 99%,NaN,NaN,None
6820,974597002,men_jeans_slim,slim_tapered_jeans,29.99,2022-04-24 11:00:00-21:30,974597,2,light_denim_blue,NaN,NaN,None,None,None
1731,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-04-24 11:00:00-21:30,690449,43,black,NaN,NaN,None,None,None
2879,938875010,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-24 11:00:00-21:30,938875,10,black,NaN,NaN,None,None,None
6620,938875014,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-24 11:00:00-21:30,938875,14,light_denim_blue,"Cotton 99%, Spandex 1%",Cotton 99%,NaN,NaN,None
5713,875105026,men_jeans_relaxed,relaxed_jeans,29.99,2022-04-24 11:00:00-21:30,875105,26,light_gray,NaN,NaN,None,None,None
2016,1008549006,men_jeans_regular,regular_jeans,19.99,2022-04-24 11:00:00-21:30,1008549,6,dark_gray,NaN,NaN,None,None,None
1463,1024256008,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,8,denim_blue,NaN,NaN,None,None,None
